In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyneb as pn

## The EmissionLine class

This is the class characterizing emission lines. An emission line is identified by an element and a spectrum (which identify the emitting ion), a wavelength in Angstrom, a blend flag, a label in the standard PyNeb format, an observed intensity, a reddening-corrected intensity, an expression describing how the intensity depends on the included wavelengths, an observational error and an error on the corrected intensity. Other programs determine one or more of these values.

To instantiate an Emission Line object, use the following:

In [2]:
line = pn.EmissionLine('O', 3, 5007, obsIntens=[1.4, 1.3])
line2 = pn.EmissionLine(label = 'O3_5007A', obsIntens=320, corrected=True)

obsIntens is a value, a list or a numpy array of values corresponding to the observed intensity of the given emission line.

In [3]:
print(line)

Line O3 O3_5007A


To know how the label of a given line is exactly spelled, you can print the dictionary pn.LINE_LABEL_LIST

In [4]:
print(pn.LINE_LABEL_LIST['O3'])

['4931A', '4959A', '5007A', '2315A', '2321A', '2331A', '4363A', '1658A', '1661A', '1666A', '2497A', '5833A', '88.3m', '32.6m', '51.8m']


It is possible to instantiate a line not contained in the pn.LINE_LABEL_LIST. In this case a warning is issued, but the code doesn't stop.

The observed intensity is stored in __line.obsIntens__ and the extinction-corrected intensity is stored in __line.corrIntens__. __line.corrIntens__ is set to 0.0 when the line is instantiated, unless the parameter corrected is set to __True__, in which case the observed value __obsIntens__ is copied to the __corrIntens__ slot (the same applies for __corrError__, which is set to __obsError__). 

The __corrIntens__ value can also be computed using an instantiation of the __pn.RedCorr__ class:

In [5]:
redcorr = pn.RedCorr(E_BV = 0.87, law = 'F99')

In [6]:
line.correctIntens(redcorr) #redcorr is used to compute line.corrIntens

The line information is printed using:

In [7]:
line.printLine()

Line O3 O3_5007A evaluated as L(5007)
Observed intensity: [1.4 1.3]
Observed error: [0. 0.]
Corrected intensity: [22.58352855 20.97041937]
Corrected error: [0. 0.]


Most of the times, users will not need to define or manipulate EmissionLine objects, since most of the work on the EmissionLine objects will be performed from the Observation class (read data, extinction correction); see next section.

WARNING: Note that the wavelengths assigned to EmissionLine objects are simply the numerical part of the label:

In [8]:
Hb1 = pn.EmissionLine(label='H1r_4861A').wave
print(Hb1)

4861.0


whereas pn.Atom computes them as the difference from energy levels:

In [9]:
Hb2 = pn.RecAtom('H', 1).getWave(4, 2)
print(Hb2)

4861.3316598713955


This can cause small errors when both methods are used simultaneously. For instance, the extinction correction at Hb1 is slightly different from the expected value of 1: 

In [10]:
rc = pn.RedCorr() 
rc.E_BV = 1.34
rc.law = 'F99'
rc.getCorrHb(Hb1)

1.000410798530041

This happens because the ExtCorr uses the precise H$\beta$ value computed from energy levels. If this roundoff error exceeds your tolerance, a possible workaround is forcing the emission line to have exactly the wavelength computed from the energy levels:

In [11]:
O3w = pn.EmissionLine('O', 3, wave=5007).wave
print(rc.getCorrHb(O3w))

0.8362020498282933


In [12]:
Hb11 = pn.EmissionLine('H', 1, wave=Hb2).wave
print(rc.getCorrHb(Hb11)) 
# This will generate a warning (as the transition is not included in the inventory for the specified atom), 
# but the code won't stop.

warng EmissionLine: Atom H1 not valid
1.0


## The Observation class: reading and dealing with observations

### Reading observation from a file

pn.Observation is the class characterizing observation records. An observation record is composed of an object identifier, the observed intensity of one or more emission lines, and, optionally, the dereddened line intensities and the identifier of the extinction law used, and the value of c(Hbeta).

Observations can be initialized by reading data files, which can be organized with different emission lines either in rows or columns (usually, in a survey of many objects with few emission lines emission lines change across columns; and in a high-resolution observation of a small sample of objects lines change across rows).

The following is an example of how to define an observation:

In [13]:
obs = pn.Observation()

In [14]:
%%writefile observations1.dat
LINE SMC_24
S4_10.5m   7.00000
Ne2_12.8m  8.3000
Ne3_15.6m 34.10
S3_18.7m  10.
O2_3726A  39.700
O2_3729A  18.600
Ne3_3869A 18.90
Ne3_3968A  6.4
S2_4069A   0.85
S2_4076A   0.450
O3_4363A   4.36
H1r_4861A 100.00
O3_5007A 435.09
N2_5755A   0.510000
S3_6312A   0.76
O1_6300A   1.69
O1_6364A   0.54
N2_6548A   6.840000
H1r_6563A  345.00
N2_6584A  19.00
S2_6716A   1.220000
S2_6731A   2.180000
Ar3_7136A  4.91
O2_7319A+   6.540000
O2_7330A+   5.17

Overwriting observations1.dat


In [15]:
obs.readData('observations1.dat', fileFormat='lines_in_rows', err_default=0.05) # fill obs with data read from smc_24.dat

In [16]:
obs.printIntens(returnObs=True)

S4_10.5m      7.000
Ne2_12.8m     8.300
Ne3_15.6m    34.100
S3_18.7m     10.000
O2_3726A     39.700
O2_3729A     18.600
Ne3_3869A    18.900
Ne3_3968A     6.400
S2_4069A      0.850
S2_4076A      0.450
O3_4363A      4.360
H1r_4861A   100.000
O3_5007A    435.090
N2_5755A      0.510
S3_6312A      0.760
O1_6300A      1.690
O1_6364A      0.540
N2_6548A      6.840
H1r_6563A   345.000
N2_6584A     19.000
S2_6716A      1.220
S2_6731A      2.180
Ar3_7136A     4.910
O2_7319A+     6.540
O2_7330A+     5.170


In [17]:
obs.extinction.law = 'CCM89'  # define the extinction law from Cardelli et al.
obs.correctData()                      # the dereddened data are computed

The data can be read by the readData method as above or directly while instantiating the object:

In [18]:
obs = pn.Observation('observations1.dat', fileFormat='lines_in_rows', corrected=True)

The format of the data file from which the emission line intensities are read can be one of three kinds: "lines_in_rows" as above, or "lines_in_cols" like this one:

In [19]:
%%writefile observations2.dat
NAME O2_3726A  O2_3726Ae O2_3729A O2_3729Ae
NGC3132 0.93000   0.05000   0.17224200 0.10  
IC418 1.28000   0.05000   0.09920000 0.05 
M33 0.03100   0.080     0.03100    0.10

Overwriting observations2.dat


In [20]:
obs2 = pn.Observation('observations2.dat', fileFormat='lines_in_cols', corrected=True)

or fileFormat='lines_in_rows_err_cols' (errors labeled “err”. Don’t name an observation “err”!) like this one:

In [21]:
%%writefile observations3.dat
LINE     TT   err  TT2 err TT3 err
cHbeta   1.2  0.0  1.5 0.2 1.1 0.2
O3_5007A 1.5  0.15 1.3  .2 1.1 0.1
H1_6563A 2.89 0.05 1.6 0.3 1.3 0.1
N2_6584A 1.   0.20 0.3 0.5 1.5 0.1

Overwriting observations3.dat


In [22]:
#obs3 = pn.Observation('observations3.dat', fileFormat='lines_in_rows_err_cols', corrected=False)

The delimiter between the columns is any sequence of spaces or TAB, but it can be changed using the delimiter parameter. The line names are defined by a label starting with the name of the atom (‘O2’), followed by an underscore, followed by a wavelength and ending with a unit (‘A’ or ‘m’). The list of all the lines managed by PyNeb, ordered by atoms, is obtained by entering:

In [23]:
for atom in pn.LINE_LABEL_LIST:
    print(atom, pn.LINE_LABEL_LIST[atom])

H1r ['1216A', '1026A', '973A', '6563A', '4861A', '4341A', '4102A', '3970A', '3889A', '3835A', '3798A', '1.87m', '1.28m', '1.09m', '9546A', '9229A']
He1r ['5876A', '2945A', '3188A', '3614A', '3889A', '3965A', '4026A', '4121A', '4388A', '4438A', '4471A', '4713A', '4922A', '5016A', '5048A', '5876A', '6678A', '7065A', '7281A', '9464A', '10830A', '11013A', '11969A', '12527A', '12756A', '12785A', '12790A', '12846A', '12968A', '12985A', '13412A', '15084A', '17003A', '18556A', '18685A', '18697A', '19089A', '19543A', '20425A', '20581A', '20602A', '21120A', '21132A', '21608A', '21617A']
He2r ['1640A', '1215A', '1084A', '4686A', '3203A', '6560A', '5411A', '4859A', '4541A', '6407A', '4198A']
3He2 ['3.50c']
Al2 ['2674A', '2670A', '2661A', '1671A', '4451A', '4463A', '4488A', '164.2m', '54.1m', '80.7m']
Ar2 ['7.0m']
Ar3 ['7136A', '7751A', '8036A', '3005A', '3109A', '3154A', '5192A', '9.0m', '6.37m', '21.8m']
Ar4 ['4740A', '4711A', '2868A', '7263A', '7332A', '2854A', '7170A', '7237A', '77.4m', '56.2m'

The presence of a trailing “e” at the end of the label points to the error associated to the line. The error is considered to be relative to the intensity (i.e., 0.05 means 5% of the intensity), unless the parameter errIsRelative is set to False. A common value for all the errors can be defined by the parameter __err_default__ (0.10 is the default value).

### Extinction correction in Observation class

Once the data have been read, they have to be corrected from extinction. An instantiation of __RedCorr()__ is available inside the __Observation__ object as __obs.extinction__.

If the data file contains __cHbeta__ or __E(B-V)__ alongside of line labels, the corresponding information on extinction is transmitted to the extinction correction object. Otherwise, the extinction parameters must be set manually; for example:

In [24]:
obs = pn.Observation('observations1.dat', fileFormat='lines_in_rows', corrected=True)
obs.extinction.cHbeta = 1.2
obs.extinction.E_BV = 0.34

An extinction law has to be specified in either case:

In [25]:
obs.extinction.law = 'F99'

To correct all the lines at once:

In [26]:
obs.correctData()

In [27]:
obs.printIntens(returnObs=True)

S4_10.5m      7.000
Ne2_12.8m     8.300
Ne3_15.6m    34.100
S3_18.7m     10.000
O2_3726A     39.700
O2_3729A     18.600
Ne3_3869A    18.900
Ne3_3968A     6.400
S2_4069A      0.850
S2_4076A      0.450
O3_4363A      4.360
H1r_4861A   100.000
O3_5007A    435.090
N2_5755A      0.510
S3_6312A      0.760
O1_6300A      1.690
O1_6364A      0.540
N2_6548A      6.840
H1r_6563A   345.000
N2_6584A     19.000
S2_6716A      1.220
S2_6731A      2.180
Ar3_7136A     4.910
O2_7319A+     6.540
O2_7330A+     5.170


In [28]:
obs.printIntens()

S4_10.5m      7.120
Ne2_12.8m     8.415
Ne3_15.6m    34.483
S3_18.7m     10.093
O2_3726A    171.242
O2_3729A     80.156
Ne3_3869A    78.134
Ne3_3968A    25.717
S2_4069A      3.320
S2_4076A      1.754
O3_4363A     15.704
H1r_4861A   310.254
O3_5007A   1289.851
N2_5755A      1.244
S3_6312A      1.662
O1_6300A      3.704
O1_6364A      1.170
N2_6548A     14.346
H1r_6563A   721.747
N2_6584A     39.607
S2_6716A      2.488
S2_6731A      4.435
Ar3_7136A     9.384
O2_7319A+    12.180
O2_7330A+     9.614


If you want the corrected line intensities to be normalized to a given wavelength, use the following:

In [29]:
obs.correctData(normWave=4861.)

The extinction correction can be determined by comparing the observed values to a theoretical ratio, as in the following:

In [30]:
obs.printIntens()

S4_10.5m      2.295
Ne2_12.8m     2.712
Ne3_15.6m    11.115
S3_18.7m      3.253
O2_3726A     55.194
O2_3729A     25.836
Ne3_3869A    25.184
Ne3_3968A     8.289
S2_4069A      1.070
S2_4076A      0.565
O3_4363A      5.062
H1r_4861A   100.000
O3_5007A    415.740
N2_5755A      0.401
S3_6312A      0.536
O1_6300A      1.194
O1_6364A      0.377
N2_6548A      4.624
H1r_6563A   232.631
N2_6584A     12.766
S2_6716A      0.802
S2_6731A      1.429
Ar3_7136A     3.025
O2_7319A+     3.926
O2_7330A+     3.099


In [31]:
obs.def_EBV(label1="H1r_6563A", label2="H1r_4861A", r_theo=2.85)
print(obs.extinction.E_BV)
obs.correctData(normWave=4861.)

[0.16483175]


In [32]:
obs.printIntens()

S4_10.5m      4.076
Ne2_12.8m     4.826
Ne3_15.6m    19.803
S3_18.7m      5.802
O2_3726A     46.576
O2_3729A     21.812
Ne3_3869A    21.722
Ne3_3968A     7.255
S2_4069A      0.950
S2_4076A      0.503
O3_4363A      4.687
H1r_4861A   100.000
O3_5007A    425.599
N2_5755A      0.454
S3_6312A      0.641
O1_6300A      1.428
O1_6364A      0.454
N2_6548A      5.657
H1r_6563A   285.000
N2_6584A     15.668
S2_6716A      0.995
S2_6731A      1.777
Ar3_7136A     3.882
O2_7319A+     5.106
O2_7330A+     4.034


By default, this method prints out the corrected intensities. To print the observed intensities, use the __returnObs=True__ parameter.

The method __getSortedLines__ returns the lines sorted in alphabetical order according to either the emitting atoms (default) or the wavelength (using the __crit='wave'__ parameter):

In [33]:
for line in obs.getSortedLines(): 
    print(line.label, line.corrIntens[0])

Ar3_7136A 3.8821497892916716
H1r_4861A 100.0
H1r_6563A 284.99999999999994
N2_5755A 0.4538465502784229
N2_6548A 5.6574680712548835
N2_6584A 15.668474852815743
Ne2_12.8m 4.826029712555136
Ne3_15.6m 19.8027022261585
Ne3_3869A 21.721862162658756
Ne3_3968A 7.2549847099308185
O1_6300A 1.4279238824617235
O1_6364A 0.4536974769399841
O2_3726A 46.57643988525366
O2_3729A 21.812058000703153
O2_7319A+ 5.106424464804535
O2_7330A+ 4.033770322547239
O3_4363A 4.687051074032947
O3_5007A 425.5991545953096
S2_4069A 0.95040451703046
S2_4076A 0.5026812314820526
S2_6716A 0.9954054122181787
S2_6731A 1.7765637886180774
S3_18.7m 5.801845911170803
S3_6312A 0.6414610121168983
S4_10.5m 4.076478721339332


The following method, which gives the list of all the atoms implied in the observed emission lines, will be useful later:

In [34]:
atomList = obs.getUniqueAtoms()

In [35]:
atomList

array(['Ar3', 'H1r', 'N2', 'Ne2', 'Ne3', 'O1', 'O2', 'O3', 'S2', 'S3',
       'S4'], dtype='<U3')

### Adding observations and lines

Once an __Observation__ object is instantiated, you can add a new observation (corresponding, e.g., to a new object or a new fiber) by using:

In [36]:
obs.addObs('test', np.random.rand(25))

where ‘test’ is the name of the new observation. The new observation must have the same size of __obs__, that is, it must contain __obs.n_lines__ lines.

In [37]:
obs.printIntens()

S4_10.5m      4.076    0.933
Ne2_12.8m     4.826    0.596
Ne3_15.6m    19.803    0.664
S3_18.7m      5.802    0.457
O2_3726A     46.576    0.339
O2_3729A     21.812    0.890
Ne3_3869A    21.722    0.600
Ne3_3968A     7.255    0.833
S2_4069A      0.950    0.878
S2_4076A      0.503    0.338
O3_4363A      4.687    0.433
H1r_4861A   100.000    0.570
O3_5007A    425.599    0.451
N2_5755A      0.454    0.263
S3_6312A      0.641    0.491
O1_6300A      1.428    0.146
O1_6364A      0.454    0.938
N2_6548A      5.657    0.723
H1r_6563A   285.000    0.700
N2_6584A     15.668    0.010
S2_6716A      0.995    0.302
S2_6731A      1.777    0.848
Ar3_7136A     3.882    0.179
O2_7319A+     5.106    0.009
O2_7330A+     4.034    0.032


In [38]:
line = pn.EmissionLine(label='Cl3_5518A', obsIntens=[3.5, 2.5])
obs.addLine(line)

In [39]:
obs.printIntens()

S4_10.5m      4.076    0.933
Ne2_12.8m     4.826    0.596
Ne3_15.6m    19.803    0.664
S3_18.7m      5.802    0.457
O2_3726A     46.576    0.339
O2_3729A     21.812    0.890
Ne3_3869A    21.722    0.600
Ne3_3968A     7.255    0.833
S2_4069A      0.950    0.878
S2_4076A      0.503    0.338
O3_4363A      4.687    0.433
H1r_4861A   100.000    0.570
O3_5007A    425.599    0.451
N2_5755A      0.454    0.263
S3_6312A      0.641    0.491
O1_6300A      1.428    0.146
O1_6364A      0.454    0.938
N2_6548A      5.657    0.723
H1r_6563A   285.000    0.700
N2_6584A     15.668    0.010
S2_6716A      0.995    0.302
S2_6731A      1.777    0.848
Ar3_7136A     3.882    0.179
O2_7319A+     5.106    0.009
O2_7330A+     4.034    0.032
Cl3_5518A     5.534    3.953


### Getting line intensities

You can extract the line intensities from an __Observation__ object by, for example:

In [40]:
obs.names

['SMC_24', 'test']

In [41]:
obs.getIntens(obsName='SMC_24')

{'S4_10.5m': 4.076478721339332,
 'Ne2_12.8m': 4.826029712555136,
 'Ne3_15.6m': 19.8027022261585,
 'S3_18.7m': 5.801845911170803,
 'O2_3726A': 46.57643988525366,
 'O2_3729A': 21.812058000703153,
 'Ne3_3869A': 21.721862162658756,
 'Ne3_3968A': 7.2549847099308185,
 'S2_4069A': 0.95040451703046,
 'S2_4076A': 0.5026812314820526,
 'O3_4363A': 4.687051074032947,
 'H1r_4861A': 100.0,
 'O3_5007A': 425.5991545953096,
 'N2_5755A': 0.4538465502784229,
 'S3_6312A': 0.6414610121168983,
 'O1_6300A': 1.4279238824617235,
 'O1_6364A': 0.4536974769399841,
 'N2_6548A': 5.6574680712548835,
 'H1r_6563A': 284.99999999999994,
 'N2_6584A': 15.668474852815743,
 'S2_6716A': 0.9954054122181787,
 'S2_6731A': 1.7765637886180774,
 'Ar3_7136A': 3.8821497892916716,
 'O2_7319A+': 5.106424464804535,
 'O2_7330A+': 4.033770322547239,
 'Cl3_5518A': 5.533838898173915}

In [42]:
obs.getIntens()['O2_7330A+']

array([4.03377032, 0.03166219])

## Using Observation to determine ionic abundances

Once the electron temperature and density are determined, it is easy to obtain the ionic abundances from a set of emission lines included in an __Observation__ object:

In [43]:
obs = pn.Observation()
obs.readData('observations1.dat', fileFormat='lines_in_rows', err_default=0.05) # fill obs with data read from observations1.dat
obs.def_EBV(label1="H1r_6563A", label2="H1r_4861A", r_theo=2.85)
obs.correctData(normWave=4861.)
Te = [10000.]
Ne = [1e3]
# Define a dictionary to hold all the Atom objects needed
all_atoms = pn.getAtomDict(atom_list=obs.getUniqueAtoms())
# define a dictionary to store the abundances
ab_dict = {}
# we  use the following lines to determine the ionic abundances
ab_labels = ['N2_6584A', 'O2_3726A', 'O3_5007A', 'S2_6716A', 
             'S3_6312A', 'Ar3_7136A', 'Ne3_3869A']
for line in obs.getSortedLines():
    if line.label in ab_labels:
        ab = all_atoms[line.atom].getIonAbundance(line.corrIntens, Te, Ne, 
                                                  to_eval=line.to_eval, Hbeta=100)
        ab_dict[line.atom] = ab

warng _ManageAtomicData: rec data not available for Ar3
warng _ManageAtomicData: atom data not available for H1
warng _ManageAtomicData: coll data not available for H1
warng _ManageAtomicData: rec data not available for Ne3
warng _ManageAtomicData: rec data not available for S2
warng _ManageAtomicData: rec data not available for S3
warng _ManageAtomicData: rec data not available for S4


In [44]:
ab_dict

{'Ar3': array([3.21287725e-07]),
 'N2': array([3.22649264e-06]),
 'Ne3': array([2.36596811e-05]),
 'O2': array([3.64228102e-05]),
 'O3': array([0.00014833]),
 'S2': array([6.71633117e-08]),
 'S3': array([1.42901804e-06])}